In [2]:
import numpy as np
import pandas as pd
import urllib.request
from ultralytics import YOLO
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

def seperate(sentence):
    sentence = str(sentence)

    result = []

    for word in api.analyze(sentence):
        for morph in word.morphs:
            result.append(morph.lex)

    return ' '.join(result)

In [73]:
vet_list = pd.read_csv('./fulldata_02_03_01_P_동물병원.csv', encoding='euc-kr').loc[:, ['영업상태명', '사업장명', '도로명전체주소']]
vet_list = vet_list.loc[vet_list['영업상태명'] == '영업/정상']

vet_list

,영업상태명,사업장명,도로명전체주소
4771,영업/정상,문경가축병원,NaN
4827,영업/정상,일곡동물병원,광주광역시 북구 양일로305번길 8 (일곡동)
4828,영업/정상,우리동물병원,광주광역시 북구 태봉로 37-6 (임동)
4829,영업/정상,참좋은 동물병원,"광주광역시 북구 하서로 371, 1층 (양산동)"
4830,영업/정상,빛고을 동물의료센터,광주광역시 북구 중문로 74 (우산동)
...,...,...,...
10029,영업/정상,서림동물병원,충청남도 서산시 공림3길 51 (예천동)
10030,영업/정상,다솜동물메디컬센터 금정점,"부산광역시 금정구 금강로 624, 1층 (남산동)"
10031,영업/정상,다솜고양이메디컬센터 금정점,"부산광역시 금정구 금강로 624, 2,3층 (남산동)"
10032,영업/정상,포레스트동물의료센터,"경기도 수원시 팔달구 팔달로 63, 103,104호 (고등동)"


In [74]:
vet_list[vet_list['도로명전체주소'].isnull()]


,영업상태명,사업장명,도로명전체주소
4771,영업/정상,문경가축병원,NaN
4906,영업/정상,현대동물병원,NaN
5254,영업/정상,금오산낙협동물병원,NaN
6270,영업/정상,원평동물병원,NaN
6274,영업/정상,성진동물병원,NaN
6449,영업/정상,봉화가축병원,NaN
6686,영업/정상,영인동물병원,NaN
7021,영업/정상,이영기수의과병원,NaN
7207,영업/정상,박동물병원,NaN
7365,영업/정상,국군의학연구소 동물병원,NaN


In [75]:
# row 생략 없이 출력
# pd.set_option('display.max_rows', None)
vet_name_label = vet_list.loc[:, ['사업장명']]
vet_name_label.rename(columns={'사업장명':'text'}, inplace=True)
vet_name_label['category'] = '__label__1.vet_name'

# vet_list_label.columns

# vet_list_label = vet_list_label.iloc[:, 1:]
vet_name_label

,text,category
4771,문경가축병원,__label__1.vet_name
4827,일곡동물병원,__label__1.vet_name
4828,우리동물병원,__label__1.vet_name
4829,참좋은 동물병원,__label__1.vet_name
4830,빛고을 동물의료센터,__label__1.vet_name
...,...,...
10029,서림동물병원,__label__1.vet_name
10030,다솜동물메디컬센터 금정점,__label__1.vet_name
10031,다솜고양이메디컬센터 금정점,__label__1.vet_name
10032,포레스트동물의료센터,__label__1.vet_name


In [76]:
vet_addr_label = vet_list.loc[:, ['도로명전체주소']]
vet_addr_label = vet_addr_label[vet_addr_label['도로명전체주소'].notnull()]
vet_addr_label.rename(columns={'도로명전체주소':'text'}, inplace=True)
vet_addr_label['category'] = '__label__2.vet_address'

vet_addr_label

,text,category
4827,광주광역시 북구 양일로305번길 8 (일곡동),__label__2.vet_address
4828,광주광역시 북구 태봉로 37-6 (임동),__label__2.vet_address
4829,"광주광역시 북구 하서로 371, 1층 (양산동)",__label__2.vet_address
4830,광주광역시 북구 중문로 74 (우산동),__label__2.vet_address
4831,광주광역시 북구 하백로5번길 1 (매곡동),__label__2.vet_address
...,...,...
10029,충청남도 서산시 공림3길 51 (예천동),__label__2.vet_address
10030,"부산광역시 금정구 금강로 624, 1층 (남산동)",__label__2.vet_address
10031,"부산광역시 금정구 금강로 624, 2,3층 (남산동)",__label__2.vet_address
10032,"경기도 수원시 팔달구 팔달로 63, 103,104호 (고등동)",__label__2.vet_address


In [77]:
treat_list = pd.read_csv('./mySaeKku_hosp_fare_list_re - mySaeKku_hosp_fare_list_re.csv', encoding='utf-8')

treat_list

,Unnamed: 0,병원이름,중분류,소분류,가격
0,1,24시 SNC 동물메디컬센터,진찰료 / 상담료필수,초진비 - 예약,"10,000원"
1,2,24시 SNC 동물메디컬센터,진찰료 / 상담료필수,초진비 - 비예약,"12,000원"
2,3,24시 SNC 동물메디컬센터,진찰료 / 상담료필수,야간 진료비 (21시~9시),"40,000원"
3,4,24시 SNC 동물메디컬센터,진찰료 / 상담료필수,재진비 - 예약,"6,000원"
4,5,24시 SNC 동물메디컬센터,진찰료 / 상담료필수,재진비 - 비예약,"8,000원"
...,...,...,...,...,...
2128,2129,대치수동물병원,슬개골 탈구 한쪽단독,5kg 이상-7kg 미만 강아지 슬개골 탈구 한쪽,상담 후 안내
2129,2130,대치수동물병원,슬개골 탈구 양쪽단독,3kg 이상-5kg 미만 강아지 슬개골 탈구 양쪽,상담 후 안내
2130,2131,대치수동물병원,슬개골 탈구 양쪽단독,5kg 이상-7kg 미만 강아지 슬개골 탈구 양쪽,상담 후 안내
2131,2132,대치수동물병원,건강검진단독,건강검진 항목,건강검진 항목표 보기 (click)


In [78]:
treat_list_label = treat_list.loc[:, ['소분류']]
treat_list_label.drop_duplicates()
treat_list_label.rename(columns={'소분류':'text'}, inplace=True)
treat_list_label['category'] = '__label__3.treat_name'

treat_list_label

,text,category
0,초진비 - 예약,__label__3.treat_name
1,초진비 - 비예약,__label__3.treat_name
2,야간 진료비 (21시~9시),__label__3.treat_name
3,재진비 - 예약,__label__3.treat_name
4,재진비 - 비예약,__label__3.treat_name
...,...,...
2128,5kg 이상-7kg 미만 강아지 슬개골 탈구 한쪽,__label__3.treat_name
2129,3kg 이상-5kg 미만 강아지 슬개골 탈구 양쪽,__label__3.treat_name
2130,5kg 이상-7kg 미만 강아지 슬개골 탈구 양쪽,__label__3.treat_name
2131,건강검진 항목,__label__3.treat_name


In [79]:
date_list = pd.read_csv('./영수증데이터 - 시트1.csv', encoding='utf-8').loc[:157, ['영수증일', '항목1 진료명', '수량', '항목1 진료비']]

treat1_list_label = date_list.loc[:, ['항목1 진료명']]
treat1_list_label.rename(columns={'항목1 진료명':'text'}, inplace=True)
treat1_list_label['category'] = '__label__3.treat_name'

date_list_label = date_list.loc[: ,['영수증일']]
date_list_label = date_list_label[date_list_label['영수증일'].notnull()]

date_list_label.rename(columns={'영수증일':'text'}, inplace=True)
date_list_label['category'] = '__label__4.date'

number_list_label = date_list.loc[:, ['수량']]
number_list_label = number_list_label[number_list_label['수량'].notnull()]
number_list_label.rename(columns={'수량':'text'}, inplace=True)
number_list_label['category'] = '__label__5.number'

price_list_label = date_list.loc[:, ['항목1 진료비']]
price_list_label.rename(columns={'항목1 진료비':'text'}, inplace=True)
price_list_label['category'] = '__label__6.price'



In [80]:
etc_list_label = pd.read_csv('./etc_list - 시트1.csv', encoding='utf-8')

In [81]:
vet_list_label = pd.concat([vet_name_label, vet_addr_label, treat_list_label, treat1_list_label, date_list_label, number_list_label, price_list_label, etc_list_label], ignore_index=True)

vet_list_label.columns

Index(['text', 'category'], dtype='object')

In [82]:
vet_list_label = vet_list_label[['category', 'text']]
vet_list_label['text'] = vet_list_label['text'].apply(seperate)

In [83]:
vet_list_label['text'] = vet_list_label['text'].astype('object')

In [10]:
vet_list_label.to_csv('./vet_list_label.txt', index=False, sep='\t')

In [7]:
vet_list_label = pd.read_csv('./vet_list_label.txt', encoding='utf-8', sep='\t')

vet_list_label

,category,text
0,__label__1.vet_name,문경 가 축 병원
1,__label__1.vet_name,일곡동물병원
2,__label__1.vet_name,우 리동물 병 원
3,__label__1.vet_name,참 좋 은 동물 병원
4,__label__1.vet_name,빛고 을 동물 의료 센터
...,...,...
13759,__label__6.price,"8 , 800"
13760,__label__6.price,"11 , 000"
13761,__label__6.price,"23 , 000"
13762,__label__6.price,"33 , 000"


In [8]:
vet_list_label['text'] = vet_list_label['text'].apply(seperate)

In [9]:
vet_list_label

,category,text
0,__label__1.vet_name,문 경 가 축 병원
1,__label__1.vet_name,일곡동물병원
2,__label__1.vet_name,우 리동물 병 원
3,__label__1.vet_name,참 좋 으 ㄴ 동물 병원
4,__label__1.vet_name,빛 고 으 ㄹ 동물 의료 센터
...,...,...
13759,__label__6.price,"8 , 800"
13760,__label__6.price,"11 , 000"
13761,__label__6.price,"23 , 000"
13762,__label__6.price,"33 , 000"


In [12]:
from gensim.models import FastText
from keras.models import load_model
import fasttext

In [17]:
text_model = fasttext.train_supervised(input="./vet_list_label.txt", label_prefix='__label__')

Read 0M words
Number of words:  10743
Number of labels: 7
Progress: 100.0% words/sec/thread:  926673 lr:  0.000000 avg.loss:  0.050102 ETA:   0h 0m 0s


In [19]:
text_model.predict(seperate('처치-염종치료 C {식모,세정 드레싱)'))

(('__label__2.vet_address',), array([    0.76801]))

In [13]:
# model1 = fasttext.train_supervised(input="./vet_list_label.txt", epoch=100, bucket=20000, lr=1, wordNgrams=6, dim=80, label_prefix='__label__')
model1 = fasttext.train_supervised(input="./vet_list_label.txt", wordNgrams=3, label_prefix='__label__')


print("결과:",model1.predict(seperate('입원-인퓨전펌프')))

Read 0M words
Number of words:  10743
Number of labels: 7


결과: (('__label__3.treat_name',), array([    0.96388]))


Progress: 100.0% words/sec/thread:  472512 lr:  0.000000 avg.loss:  0.065774 ETA:   0h 0m 0s


In [14]:
print("결과:",model1.predict(seperate('정맵카테터')))
# seperate('정맵카테터')

결과: (('__label__3.treat_name',), array([    0.58491]))


In [90]:
model1.save_model('receipts_fasttext_240723')

In [2]:
from PyKomoran import Komoran, DEFAULT_MODEL

komoran = Komoran(DEFAULT_MODEL['FULL'])

In [3]:
komoran.morphes('이것은 테스트')

[]

In [9]:
from khaiii import KhaiiiApi

In [9]:
api = KhaiiiApi()

for word in api.analyze('입원-인퓨전펌프'):
    for morph in word.morphs:
        print(morph.lex)


입원
-
인퓨전
펌프


In [24]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

def seperate(sentence):
    sentence = str(sentence)

    result = []

    for word in api.analyze(sentence):
        for morph in word.morphs:
            result.append(morph.lex)

    return ' '.join(result)

In [25]:
df = vet_list_label['text'].apply(seperate)

df

0                              문경 가 축 병원
1                                 일곡동물병원
2                              우 리동물 병 원
3                            참 좋 은 동물 병원
4                          빛고 을 동물 의료 센터
                      ...               
13241      2024 년 6 월 7 일 오후 2 : 38 : 41
13242     2024 년 6 월 12 일 오후 4 : 49 : 05
13243      2024 년 6 월 8 일 오후 2 : 59 : 58
13244     2024 년 5 월 29 일 오후 2 : 40 : 52
13245    2024 년 6 월 15 일 오전 10 : 54 : 25
Name: text, Length: 13246, dtype: object